In [ ]:
import pathlib
import polars as pl

datadir = pathlib.Path.cwd().parent / "data"

pl_hotel       = pl.read_parquet(datadir / "hotel.parquet")
pl_customer    = pl.read_parquet(datadir / "customer.parquet")
pl_reservation = pl.read_parquet(datadir / "reservation.parquet")
pl_campaign    = pl.read_parquet(datadir / "campaign.parquet")

# 7章 結合


## 7-1 1対1または多対1の関係のテーブルの結合
### Q: ビジネスホテルかつ宿泊人数が1名の予約履歴の抽出


#### Awesome

In [ ]:
(
    pl_reservation
    #（1）people_numが1人のデータのみ抽出
    .filter(pl.col("people_num") == 1)
    #（3）ビジネスホテルのみのマスタを内部結合
    .join(
        #（2）pl_hotelからビジネスホテルの行のみ抽出
        pl_hotel.filter(pl.col("hotel_type") == "ビジネスホテル").select("hotel_id"),
        on="hotel_id", how="inner"
    )
)

#### Awesome

In [ ]:
(
    pl_reservation
    #（1）people_numが1人のデータのみ抽出
    .filter(pl.col("people_num") == 1)
    #（3）ビジネスホテルのみのマスタをセミ結合
    .join(
        #（2）pl_hotelからビジネスホテルの行のみ抽出
        pl_hotel.filter(pl.col("hotel_type") == "ビジネスホテル"),
        on="hotel_id", how="semi"
    )
)

## 7-2 1対多の関係のテーブルの結合
### Q: ホテルマスタにホテルの売上と予約数を付与


#### Awesome

In [ ]:
(
    pl_hotel
    #（2）hotel_id列を結合キーとして（1）の集計結果を左外部結合
    .join(
        #（1）pl_reservationから未キャンセルかつcheckout_dateの年が2019の行のみ抽出し、
        #    hotel_idを集約キーとしてgroup by集計
        pl_reservation
        .filter((pl.col("status") != "canceled")
                & (pl.col("checkout_date").dt.year() == 2019))
        .group_by("hotel_id").agg([
            pl.col("total_price").sum(),
            pl.len()
        ]),
        on="hotel_id", how="left"
    )
    #（3）結合されなかった行の数値を0埋め
    .with_columns(
        total_price=pl.col("total_price").fill_null(0),
        len=pl.col("len").fill_null(0)
    )
)

## 7-3 多対多の関係のテーブルの結合
### Q: 顧客マスタに対して、顧客のホテル種別ごとの予約数を付与


#### Awesome

In [ ]:
(
    pl_customer
    #（3）pl_customerに（1）〜（2）の集計結果を左外部結合
    .join(
        pl_reservation
        #（1）pl_reservationにpl_hotelを左外部結合
        .join(pl_hotel.select(["hotel_id", "hotel_type"]), on="hotel_id", how="left")
        #（2）customer_idごとにホテル種別ごとのデータ数をカウント
        .group_by("customer_id").agg([
            pl.col("reservation_id").filter(pl.col("hotel_type") == "旅館")
                .len().alias("ryokan_cnt"),
            pl.col("reservation_id").filter(pl.col("hotel_type") == "リゾートホテル")
                .len().alias("resort_hotel_cnt"),
            pl.col("reservation_id").filter(pl.col("hotel_type") == "ビジネスホテル")
                .len().alias("business_hotel_cnt"),
            pl.col("reservation_id").filter(pl.col("hotel_type") == "民宿")
                .len().alias("minsyuku_cnt")
        ]),
        on="customer_id", how="left"
    )
    #（4）結合されなかった行の数値を0埋め
    .with_columns(
        ryokan_cnt=pl.col("ryokan_cnt").fill_null(0),
        resort_hotel_cnt=pl.col("resort_hotel_cnt").fill_null(0),
        business_hotel_cnt=pl.col("business_hotel_cnt").fill_null(0),
        minsyuku_cnt=pl.col("minsyuku_cnt").fill_null(0)
    )
)

## 7-4 すべての結合の組み合わせの生成
### Q: 顧客ごとの月別の売上を計算（売上のない月も出力）


#### Awesome


In [ ]:
from datetime import date
(
    pl_customer.select("customer_id")
    #（2）pl_customerに対して月の時系列をクロス結合し、customer_idごとに全ての月の行を生成
    .join(
        #（1）1ヶ月間隔の時系列を生成
        pl.date_range(date(2019, 1, 1), date(2019, 12, 1), "1mo", eager=True)
        .alias("month").to_frame(),
        how="cross"
    )
    #（5）（2）のデータに対して（3）〜（4）の集計結果を左外部結合
    .join(
        pl_reservation
        #（3）未キャンセルデータを抽出
        .filter(pl.col("status") != "canceled")
        #（4）customer_idとmonthごとにtotal_priceの総和を計算
        .group_by([
            "customer_id",
            pl.col("checkout_date").dt.truncate("1mo").dt.date().alias("month")
        ])
        .agg(pl.col("total_price").sum()),
        on=["customer_id", "month"], how="left"
    )
    #（6）結合されなかった行の数値を0埋め
    .with_columns(pl.col("total_price").fill_null(0))
)

## 7-5 不等式条件での結合
### Q: 予約履歴データにキャンペーン情報を付与


#### Not Awesome


In [ ]:
(
    pl_reservation
    #（1）pl_reservationにpl_campaignをクロス結合
    .join(pl_campaign, how="cross")
    #（2）pl_reservationのreserved_atがpl_campaignのstarts_atとends_atの間にある行のみ抽出
    .filter(pl.col("reserved_at").is_between(pl.col("starts_at"), pl.col("ends_at")))
    #（3）不要な列を削除
    .drop(["starts_at", "ends_at"])
)

In [ ]:
(
    pl_reservation
    #（1）reserved_atを日付単位に丸めたreserve_date列を作成
    .with_columns(reserve_date=pl.col("reserved_at").dt.date())
    #（4）reserve_dateを結合キーとしてキャンペーンマスタと結合
    .join(
        pl_campaign
        #（2）starts_atとends_atの間の日付単位の時系列を作成
        .select([
            pl.date_ranges(
                pl.col("starts_at").dt.date(), pl.col("ends_at").dt.date(), "1d"
            )
            .alias("reserve_date"),
            pl.col("campaign_name")
        ])
        #（3）日付の時系列を複数行に展開
        .explode("reserve_date"),
        on="reserve_date", how="left"
    )
    #（5）不要な列を削除
    .drop("reserve_date")
)